# Investment

## 1. Data/settings

In [1]:
try:
    t = t
except NameError:
    clean_up = True
    %run StdPackages.ipynb
    name, t = 'full', 2019
    ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
    with open(f"{d['processedData']}\\glob_{t}","rb") as file: # load global settings anmed glob_name
        glob=pickle.load(file)
    db_IO = GpyDB(pickle_path = os.path.join(d['processedData'], f'{name}_{t}')) # load IO database named IO_name
    addTimeToDb(db_IO, glob.db['t'].vals, exceptions = ['sigma'])
module = '_'.join([name,str(t),'I'])
db_IO0 = db_IO.copy() # create copy

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.


## 2. Initial values

Set up nesting tree using the nesting structure:

In [2]:
tree = nestingTree.tree(name = f'{module}_ces', tree = db_IO.get('nestInvestment').to_list()) # individual tree
Tree = nestingTree.aggTree(name = module, trees = {tree.name: tree})(namespace = {str(n)+'_input': n for n in db_IO.get('n')})

Initialize production module, without any durables at first:

In [3]:
I = mProduction.Production(tree=Tree, glob = glob) # initialize module from nesting tree and global settings
aggregateDB.subset_db(db_IO0, Tree.db.get('s')) # goes through all symbols in db_IO and only keep the elements that are in the set 's' from Tree.db
aggregateDB.subset_db(db_IO0, Tree.get('n')) # goes through all symbols in db_IO and only keep the elements that are in the set 'n' from Tree.db
robust.robust_merge_dbs(I.s.db, db_IO0, priority = 'second') # Merge IO data into the database of the module; if a symbol is in both, prioritize records from the second database

Add value shares:

In [4]:
v = valueShares.valueShares(Tree, db_IO0)
v.compile() # set up model structure, and make sure to initialize symbols if they are not yet defined in the database 
v.write(); # write GAMS code used for the model
m = v.run(exportTo = d['work'],ws=ws) # solve the "model".

Use value shares to initialize variables:
* Outputs and inputs are provided by IO data.
* For intermediate goods, assume a price of 1 (default option in the class, so we don't have to do anything) and set value share = quantity.
* Set share parameters to the ones identified by value share system.

In [5]:
gpyDB.GpyDBs_AOM_Second(I.s.db, gpy(adj.rc_pd(m.out_db.get('vD'), I.get('int')).rename('qD'))) # set intermediate goods levels
gpyDB.GpyDBs_AOM_Second(I.s.db, gpy(m.out_db.get('mu').xs(I.get('t0')[0]).rename('mu'))) # set intermediate goods levels

### 3. Static calibration

In [6]:
I.compile(initDB=True) # set up model structure, and make sure to initialize symbols if they are not yet defined in the database (initDB = True)
I.s.setstate('C') # set to calibration state
I.write(); # write GAMS code
mStatic = I.run(exportTo = d['work'],ws=ws,**{'cns': 'CONOPT4'}) # solve the model using CONOPT4.

### 4. Export 

In [7]:
[gpyDB.GpyDBs_AOM_Second(I.s.db, mStatic.out_db[k]) for k in I.s.db.symbols];
I.s.setstate('B')

Export:

In [8]:
with open(os.path.join(d['modules'], I.name), "wb") as file:
    pickle.dump(I,file)